In [1]:
# IR PS3
import re
import string

In [2]:
f_time_all = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.ALL", 'r')
f_query = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.QUE", 'r')
f_stp = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.STP", 'r')

In [3]:
def parse_documents(f):
    documents = []
    fstr = f.read()
    regex = r"\*TEXT\s+\d{3}\s+\d{2}/\d{2}/\d{2}\s+PAGE\s+\d{3}\n\n"
    
    iters = []
    result = re.finditer(regex, fstr, re.DOTALL)
    for key in result:
        iters.append(key)

    for i in range(len(iters)-1):
        i1 = iters[i].span()[1]
        i2 = iters[i+1].span()[0]
        
        documents.append(fstr[i1:i2])
    
    return documents

In [4]:
def parse_queries(f):
    text = f.read()
    result = re.findall(r'FIND\s+\d+\s+(.*)', text)
    
    return result

In [5]:
def parse_stopwords(f):
    text = f.read()
    text = text.lower()
    
    return set(nltk.word_tokenize(text))

In [6]:
docs = parse_documents(f_time_all)

In [7]:
queries = parse_queries(f_query)

In [8]:
samp_doc = docs[0]

In [9]:
stopwords = parse_stopwords(f_stp)

NameError: name 'nltk' is not defined

In [ ]:
# Preprocessing
# 1. conv. to lowercase
# 2. tokenization
# 3. stopword, spl char removal
# 4. stemming

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP.CS2K16.001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TEMP.CS2K16.001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords and i not in string.punctuation:
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
transform_text(samp_doc)[:251]

In [ ]:
processed_docs = []

for doc in docs:
    processed_docs.append(transform_text(doc))

In [ ]:
# 1. Cosine Similarity

In [ ]:
# Wik = (TFik/LENi)*log(N+1)/0.5+DFk
# TFik - term frequency within doc.
# LENi - len of current doc.
# N - total docs
# DFk - document frequency of the word (no. of documents for which the word exists)

In [ ]:
import math

In [ ]:
def TF(word, mymap):
    return mymap[word]

def DF(word, corpus_maps):
    count = 0
    
    for hm in corpus_maps:
        if word in hm:
            count += 1
    
    return count

def calculate_weights(doc, doc_map, corpus_maps):
    vector = [0]*len(doc_map)
    
    mymap = {}
    
    for i in doc.split():
        if i in mymap:
            mymap[i] +=1
        else:
            mymap[i] = 1
    
    for word in doc.split():
        TFik = TF(word, mymap)
        LENi = len(doc)
        N = len(corpus_maps)
        DFk = DF(word, corpus_maps)
        
        try:
            index = list(doc_map.keys()).index(word)
            vector[index] = (TFik/LENi)*(math.log(N+1)/(0.5+DFk))
        except ValueError:
            pass
    
    return vector

In [ ]:
doc_map = {}

for doc in processed_docs:
    for word in doc.split():
        if word in doc_map:
            doc_map[word] += 1
        else:
            doc_map[word] = 1

doc_matrix = []

list_of_maps = []

for doc in processed_docs:
    hm = {}
    
    for word in doc.split():
        if word in hm:
            hm[word] += 1
        else:
            hm[word] = 1
    
    list_of_maps.append(hm)

for doc in processed_docs:
    vector = calculate_weights(doc, doc_map, list_of_maps)
    doc_matrix.append(vector)

In [ ]:
print(doc_matrix)